In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
from scipy import constants as const
import plotHelperLatex
plotHelperLatex.setMatplotSettings()
#plt.rcParams['text.usetex'] = True

#Sellmeier equation
def n_squared(lamb, coeffs):
    '''lamb in µm'''
    n2 = 1
    lsquare = lamb**2
    for index in range(3):
        n2 = n2 + coeffs[index, 0]*lsquare/(lsquare-coeffs[index, 1]**2)
    return n2

#Lensmaker equation
def lensmaker(n, R1, R2, d=0):
    '''Returns 1/focal length'''
    f_inv = (n-1)*(1/R1 - 1/R2 + (n-1)*d/(n*R1*R2))
    return f_inv


def wavToEnergy(lamb):
    #Energy in eV, lamb in nm
    return const.h*const.c/lamb*1e9/const.elementary_charge

def energyToWav(E):
    #Energy in eV, lamb in nm
    return const.h*const.c/E*1e9/const.elementary_charge



#https://refractiveindex.info/?shelf=glass&book=fused_silica&page=Malitson
#Maltison 1965 UV fused silica: 0.21 µm - 3.71 µm
coeffsUVFS = np.array([[0.6961663, 0.0684043], [0.4079426, 0.1162414], [0.8974794, 9.896161]])

#Energy range is 1.3 eV to 5 eV in .1 eV steps
energies = np.linspace(1.3, 5, 37)*const.elementary_charge
wavs = const.h*const.c/energies


fig1, axs1 = plt.subplots(1,1, layout='constrained', figsize = plotHelperLatex.figSizer(1, 2.2), dpi = 144)
#fig2, axs2 = plt.subplots(1,1, layout='constrained', figsize = (6,4), dpi=200)
#µm for sellmeier
n_wav = np.sqrt(n_squared(wavs*1e6, coeffsUVFS))
axs1.plot(wavs*1e9, n_wav)
axs1.set_ylabel('n / 1')
axs1.set_xlabel(r'$\lambda$ / nm')
axs1.set_xticks(np.arange(250, 951, 100))
secax0 = axs1.secondary_xaxis('top', functions=(wavToEnergy, energyToWav))
secax0.set_xticks([1.5, 2, 3, 4, 5])
secax0.set_xlabel("E / eV")

#second part with focal distance
R1 = 59.5
yaxisNtoF = lambda n_in: -1/lensmaker(n_in, np.inf, R1)
#only for the particular case of plano conves
yaxisFtoN = lambda f_in: (1+R1/f_in)

print(yaxisNtoF(yaxisFtoN(100)))
focal_distance = -lensmaker(n_wav, np.inf, R1)
focal_distance = 1/focal_distance
#axs1.plot(wavs*1e9, yaxisFtoN(focal_distance), 'r')
#axs1.set_ylabel('f / mm')
#axs1.set_xlabel(r'$\lambda$ / nm')
#axs1.set_xticks(np.arange(250, 951, 100))



secax1 = axs1.secondary_yaxis('right', functions=(yaxisNtoF, yaxisFtoN))
#secax1.set_xticks([1.5, 2, 3, 4, 5])
secax1.set_ylabel('f / mm')

plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import plotHelperLatex
plotHelperLatex.setMatplotSettings()

#taken from edmund optics: Laser Optics and Resource Guide Section 2: Gaussian Beam Propagation

w_c = lambda w0, lamb, z: w0*np.sqrt(1+(lamb*z/(np.pi*w0**2))**2)
w0_c = lambda lamb, theta: lamb/np.pi/theta
zr_c = lambda lamb, w0: np.pi*w0**2/lamb

#collimated beam through lens with focal distance f
#taken from angular aperture
theta_c = lambda D, f: np.arctan(D/(2*f))



wav = 653e-9 # nm
#https://refractiveindex.info/?shelf=glass&book=fused_silica&page=Malitson
#Maltison 1965 UV fused silica: 0.21 µm - 3.71 µm
coeffsUVFS = np.array([[0.6961663, 0.0684043], [0.4079426, 0.1162414], [0.8974794, 9.896161]])
n = np.sqrt(n_squared(wav*1e6,coeffsUVFS))
f = 1/lensmaker(n, 59.4, np.inf)*1e-3

diameters = np.array([1, 2, 3, 4])*1e-3 # mm
thet = theta_c(diameters, f)
w0 = w0_c(wav, thet)
#print(w0*2)

zr = zr_c(wav, w0[1])

z = np.linspace(-5e-4, zr/2, 200)


#plt.tight_layout()

fig1, axs1 = plt.subplots(1,1, layout='constrained', figsize = plotHelperLatex.figSizer(2,1.4), dpi=144)
fig2, axs2 = plt.subplots(1,1, layout='constrained', figsize = plotHelperLatex.figSizer(1.6, 1.4), dpi=144)
#fig.suptitle("653 nm beam waist")
labelz = z[0]
for index in range(len(diameters)):
    axs1.plot(z*1e3, w_c(w0[index], wav, z)*2e6, label="%.1f mm" %(diameters[index]*1e3))
    axs1.text(labelz*1e3-0.65, w_c(w0[index], wav, labelz)*2e6, s="%.1f mm" %(diameters[index]*1e3), fontdict = {"fontsize": "small", "va": "center", "color": fig1.gca().lines[-1].get_color()})
axs1.set_xlabel("z / mm")
axs1.set_ylabel("beam waist / µm")
axs1.set_xlim((-1.2, 2))
axs1.yaxis.tick_right()
axs1.yaxis.set_label_position('right')



z = np.linspace(-2e-4, zr/4, 200)

labelz = z[-1]
for index in range(len(diameters)):
    axs2.plot(z*1e3, 1/(w_c(w0[index], wav, z)/w0[index])**2, label="%.1f mm" %(diameters[index]*1e3))
    axs2.text(labelz*1e3+0.01, 1/(w_c(w0[index], wav, labelz)/w0[index])**2, s="%.1f mm" %(diameters[index]*1e3), fontdict = {"fontsize": "small", "va": "center", "color": fig2.gca().lines[-1].get_color()})
axs2.set_xlabel("z / mm")
axs2.set_ylabel("relative maximum intensity / a.u.")
axs2.set_xlim((-0.3,1.1))

plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import plotHelperLatex
plotHelperLatex.setMatplotSettings()

#taken from edmund optics: Laser Optics and Resource Guide Section 2: Gaussian Beam Propagation

w_c = lambda w0, lamb, z: w0*np.sqrt(1+(lamb*z/(np.pi*w0**2))**2)
w0_c = lambda lamb, theta: lamb/np.pi/theta
zr_c = lambda lamb, w0: np.pi*w0**2/lamb

#collimated beam through lens with focal distance f
#taken from angular aperture
theta_c = lambda D, f: np.arctan(D/(2*f))


dia = 2.5e-3 # m
wavs = np.array([250,300, 400, 500, 600, 700, 800, 900])*1e-9 #wavs in m
n = np.sqrt(n_squared(wav*1e6,coeffsUVFS))
f = 1/lensmaker(n, 59.4, np.inf)*1e-3
thet = theta_c(dia, f)
w0 = w0_c(wavs, thet)

zr = np.max(zr_c(wavs, w0))

z = np.linspace(-2e-4, zr/4, 200)
zlabel = z[-1]
plt.figure()
fig1, axs1 = plt.subplots(1,1, layout='constrained', figsize = plotHelperLatex.figSizer(2,1.4), dpi=144)
#plt.title("Beam waist vs wavelength behavior at 2 mm collimated waist")
for index in range(len(wavs)):
    axs1.plot(z*1e3, w_c(w0[index], wavs[index], z)*2e6, label="%.1f nm" %(wavs[index]*1e9))
    axs1.text(zlabel*1e3+0.01, w_c(w0[index], wavs[index], zlabel)*2e6, s="%.0f nm" %(wavs[index]*1e9), fontdict = {"fontsize": "small", "va": "center", "color": plt.gca().lines[-1].get_color()})
axs1.set_xlabel("z / mm")
axs1.set_ylabel("beam waist / µm")
axs1.set_xlim((-0.2, 1.05))
#plt.legend()



plt.show()

### Overlap normalised by max pump

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
plt.rcParams['text.usetex'] = True

def gaussian(x, sig = 1, mu = 0):
    return 1/(np.sqrt(2*np.pi)*sig)*np.exp(-(x-mu)**2/(2*sig**2))

def gaussProd(x, sig1, sig2, mu1, mu2):
    mu_n = (mu1*sig2**2 + mu2*sig1**2)/(sig1**2 + sig2**2)
    sig_n = sig1*sig2/np.sqrt(sig1**2 + sig2**2)
    return 1/(np.sqrt(2*np.pi)*sig_n)*np.exp(-(x-mu_n)**2/(2*sig_n**2))

def cFactor(sigPump, sig2, mu1, mu2):
    return np.sqrt((sigPump**2 + sig2**2)/sigPump**2)*np.exp((mu1-mu2)**2/(2*(sigPump**2+sig2**2)))



def plotVisGaussians(x, sig2, dmu, norm = True, ax=None, colors = ["blue", "royalblue"]):
    if ax == None:
        fig, ax = plt.subplots(1,1)
    #probe
    #ax.plot(x, gaussian(x, 1, -dmu/2), label = r"probe: $\sigma$ = 1; $\mu = %.1f$" %(-dmu/2))
    #pump r"pump: $\sigma$ = %d; $\mu = %.1f$" %(sig2, dmu)
    ax.plot(x, gaussian(x, sig2, dmu), label = r"pump", color = colors[0], linestyle="dotted")
    #overlap
    if norm == True:
        ax.plot(x, gaussProd(x, 1, sig2, -dmu, 0), "r--", label = r"overlap")
    elif norm == False:
        relativeGaussiantoProbe = gaussian(x,1,0)*gaussian(x, sig2, dmu)/max(gaussian(x, sig2, +dmu))
        #plt.plot(x, relativeGaussiantoProbe, "r--", label = r"overlap")
        ax.fill_between(x, relativeGaussiantoProbe, hatch='/', color=colors[1], facecolor="None")
    ax.legend()
    ax.set_xlabel(r"x / $\sigma_{probe}$")
    ax.set_ylabel(r"I / a.u.")
    ax.set_title(r"$\frac{\sigma_{pump}}{\sigma_{probe}} = %.1d$, $\Delta\mu$ = %.1f $\sigma_{probe}$,  C = %.2f" %(sig2,dmu, cFactor(sig2, 1, dmu,0)), pad=8)
    #plt.show()
    return ax


x = np.linspace(-3, 3, 200)
fig, axs = plt.subplots(3,1, figsize = plotHelperLatex.figSizer(1,1.4), dpi=288, sharex=True)
for ver in range(3):
    #plot probe first
    axs[ver].plot(x, gaussian(x, 1, 0), label = r"probe", color="teal")


#case 1 µ1 = µ2, sig 1 >> sig 2
#
sig2 = 1 #times sig 1; sig2 is pump
dmu = 1.5
plotVisGaussians(x, sig2, dmu, False, ax= axs[1])

sig2 = 1 #times sig 1; sig2 is pump
dmu = 0
plotVisGaussians(x, sig2, dmu, False, colors = ["orangered", "orangered"], ax=axs[0])

sig2 = 5 #times sig 1; sig2 is pump
dmu = 0
plotVisGaussians(x, sig2, dmu, False, ax= axs[2], colors=["purple", "purple"])
fig.subplots_adjust(hspace=0)
plt.tight_layout()

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import plotHelperLatex
plotHelperLatex.setMatplotSettings()

def gaussian(x, sig = 1, mu = 0):
    return 1/(np.sqrt(2*np.pi)*sig)*np.exp(-(x-mu)**2/(2*sig**2))

def gaussProd(x, sig1, sig2, mu1, mu2):
    mu_n = (mu1*sig2**2 + mu2*sig1**2)/(sig1**2 + sig2**2)
    sig_n = sig1*sig2/np.sqrt(sig1**2 + sig2**2)
    return 1/(np.sqrt(2*np.pi)*sig_n)*np.exp(-(x-mu_n)**2/(2*sig_n**2))

def cFactor(sigPump, sig2, mu1, mu2):
    return np.sqrt((sigPump**2 + sig2**2)/sigPump**2)*np.exp((mu1-mu2)**2/(2*(sigPump**2+sig2**2)))



def plotVisGaussians(x, sig2, dmu):
    plt.figure()
    #probe
    plt.plot(x, gaussian(x, 1, -dmu/2), label = r"probe: $\sigma$ = 1; $\mu = %.1f$" %(-dmu/2))
    #pump
    plt.plot(x, gaussian(x, 2, +dmu/2), label = r"pump: $\sigma$ = %d; $\mu = %.1f$" %(sig2, dmu/2))
    #overlap
    plt.plot(x, gaussProd(x, 1, sig2, -dmu/2, dmu/2), "r--", label = r"overlap")
    plt.legend()
    plt.xlabel(r"x / $\sigma$")
    plt.ylabel(r"I / a.u.")
    plt.title(r"C = %.2e" %(cFactor(sig2, 1, dmu,0)))
    plt.show()

def plotCompareGaussians(x, sigPump, dmu_vec):
    fig = plt.figure()
    #plot pump gaussian
    plt.plot(x, gaussian(x, sigPump, 0))
    for ind in range(len(dmu_vec)):
        plt.plot(x, gaussian(x, 1, dmu_vec[ind]), label="C = %.3f" %cFactor(sigPump, 1, 0, dmu_vec[ind]))

    plt.legend()
    plt.show()
    fig.clear()

#case 1 µ1 = µ2, sig 1 >> sig 2
#
sig2 = 3 #times sig 1; sig2 is pump
dmu = 0 

x = np.linspace(-3, 3, 200)
dmu = [0, 0.1, 0.3, 0.5]
sigPump = 3
plotCompareGaussians(x, sigPump, dmu*sigPump)
sigPump = 2
plotCompareGaussians(x, sigPump, dmu*sigPump)
sigPump = 1
plotCompareGaussians(x, sigPump, dmu*sigPump)



In [ ]:
#Test binning of numerical correction factor
import numpy as np
from matplotlib import pyplot as plt
import plotHelperLatex
plotHelperLatex.setMatplotSettings()

def gaussian(x, sig = 1, mu = 0):
    return 1/(np.sqrt(2*np.pi)*sig)*np.exp(-(x-mu)**2/(2*sig**2))

def cFactor(sigPump, sig2, mu1, mu2):
    return np.sqrt((sigPump**2 + sig2**2)/sigPump**2)*np.exp((mu1-mu2)**2/(2*(sigPump**2+sig2**2)))

def binMap(map, xBinning, yBinning=0):
    '''N_binning means that N*N pixels are binned together\n
    only takes square z*z maps'''
    if yBinning == 0:
        yBinning = xBinning
    xBins = int(np.shape(map)[0]//xBinning) #along one axis
    yBins = int(np.shape(map)[1]//yBinning) #along one axis
    outmap = map.reshape(xBins, xBinning, yBins, yBinning).sum(3).sum(1)
    return outmap


#using identical gaussians for this
points_per_sig = 1024 #starting value 1000 points for 1 sig distance
sig = 1 #does not matter since I do binning in steps of sig at first
mu = 0
###----analytical cFactor ----###
cFactorAnalytic = cFactor(sig, sig, mu, mu)**2
print("Analytic correction factor:\n " + str(cFactorAnalytic))
###----numeric cFactor ----###
nsig = 5 # how far I measure to the sides
x_like = np.linspace(-nsig*sig,nsig*sig, nsig*points_per_sig)
Xvec, Yvec = np.array(np.meshgrid(x_like,x_like))


gaussStd = gaussian(Xvec, sig, mu)*gaussian(Yvec, sig, mu)



n_steps = 11
binning_steps=np.zeros(n_steps, dtype=int)

for i in range(n_steps):
    binning_steps[i] = 2**i
#print(binning_steps)

#first without added noise
print("Correction Factor numeric, no noise:")
noNoiseGaussCorrection = np.zeros(len(binning_steps))
for ind in range(len(binning_steps)):
    gaussBinned = binMap(gaussStd, binning_steps[ind])
    #pump = np.ones(np.shape(gaussBinned))
    pump = gaussBinned
    probe = gaussBinned
    denominator = np.sum(np.multiply(pump, probe)[:])
    constantPumpIntensity = np.max(pump)
    enumerator = np.sum(constantPumpIntensity*probe[:])
    #this is missing various constant factors, since this is not using the gaussian shape, but is discretized
    noNoiseGaussCorrection[ind] = enumerator/denominator
    print("binning = 1/"+str(binning_steps[ind])+": " + str(noNoiseGaussCorrection[ind]))

plt.figure(dpi = 288, figsize=plotHelperLatex.figSizer(1,2))
plt.plot(np.linspace(n_steps-1,0,n_steps), noNoiseGaussCorrection, label = "no noise")
#with added noise
Noiselevels = [0.05,0.1,0.2]
NoiseGaussCorrection = np.zeros((len(Noiselevels), len(binning_steps)))
for indexSNR, SNR in enumerate(Noiselevels):

    gauss1 = gaussStd + (np.random.rand(np.shape(gaussStd)[0], np.shape(gaussStd)[1])-0.5)*np.max(gaussStd)*SNR
    gauss2 = gaussStd + (np.random.rand(np.shape(gaussStd)[0], np.shape(gaussStd)[1])-0.5)*np.max(gaussStd)*SNR
    
    for ind in range(len(binning_steps)):
        pump = binMap(gauss1, binning_steps[ind])
        probe = binMap(gauss2, binning_steps[ind])
        denominator = np.sum(np.multiply(pump, probe)[:])
        constantPumpIntensity = np.max(pump)
        enumerator = np.sum(constantPumpIntensity*probe[:])
        #this is missing various constant factors, since this is not using the gaussian shape, but is discretized
        NoiseGaussCorrection[indexSNR, ind] = enumerator/denominator
    plt.plot(np.linspace(n_steps-1,0,n_steps), NoiseGaussCorrection[indexSNR], label = "SNR = %.2f dB" %(10*np.log10(1/SNR)), linestyle='dashed')



plt.plot([0,n_steps-1], [2,2], label="analytical correction", linestyle="dotted", color = 'red')
plt.xlabel(r"$\mathrm{log_2}$ of discrete point density per $\sigma$ / 1")
plt.ylabel("Correction factor / 1")
plt.ylim([1,2.5])
plt.legend(loc="lower right", fontsize = "small")

In [ ]:
#Test binning of numerical correction factor
#Testing how the offset actually behaves
import numpy as np
from matplotlib import pyplot as plt
import plotHelperLatex
plotHelperLatex.setMatplotSettings()

def gaussian(x, sig = 1, mu = 0):
    return 1/(np.sqrt(2*np.pi)*sig)*np.exp(-(x-mu)**2/(2*sig**2))

def cFactor(sigPump, sig2, mu1, mu2):
    return np.sqrt((sigPump**2 + sig2**2)/sigPump**2)*np.exp((mu1-mu2)**2/(2*(sigPump**2+sig2**2)))

def binMap(map, xBinning, yBinning=0):
    '''N_binning means that N*N pixels are binned together\n
    only takes square z*z maps'''
    if yBinning == 0:
        yBinning = xBinning
    xBins = int(np.shape(map)[0]//xBinning) #along one axis
    yBins = int(np.shape(map)[1]//yBinning) #along one axis
    outmap = map.reshape(xBins, xBinning, yBins, yBinning).sum(3).sum(1)
    return outmap


#using identical gaussians for this
points_per_sig = 1024 #starting value 1000 points for 1 sig distance
sig = 1 #does not matter since I do binning in steps of sig at first
dmu = 1
###----analytical cFactor ----###
#one sigma multiplicator
sigPumpYModifier= 2
sigPumpXModifier = 0.8
cFactorAnalytic = cFactor(sigPumpXModifier*sig, sig, dmu/2, -dmu/2)*cFactor(sigPumpYModifier*sig, sig, dmu/2, -dmu/2)
print("Analytic correction factor:\n " + str(cFactorAnalytic))
###----numeric cFactor ----###
nsig = 5 # how far I measure to the sides
x_like = np.linspace(-nsig*sig,nsig*sig, nsig*points_per_sig)
Xvec, Yvec = np.array(np.meshgrid(x_like,x_like))


n_steps = 11
binning_steps=np.zeros(n_steps, dtype=int)
for i in range(n_steps):
    binning_steps[i] = 2**i

d_offset = [0,0.1,0.2,0.5]
GaussCorrection = np.zeros((len(d_offset),len(binning_steps)))
for indOffset, offset in enumerate(d_offset):

    pumpGauss = gaussian(Xvec, sigPumpXModifier*sig, +dmu/2+offset)*gaussian(Yvec, sigPumpYModifier*sig, +dmu/2)
    probeGauss = gaussian(Xvec, sig, -dmu/2+offset)*gaussian(Yvec, sig, -dmu/2)


    for ind in range(len(binning_steps)):
        pump = binMap(pumpGauss, binning_steps[ind])
        probe = binMap(probeGauss, binning_steps[ind])
        denominator = np.sum(np.multiply(pump, probe)[:])
        constantPumpIntensity = np.max(pump)
        enumerator = np.sum(constantPumpIntensity*probe[:])
        #this is missing various constant factors, since this is not using the gaussian shape, but is discretized
        GaussCorrection[indOffset, ind] = enumerator/denominator


plt.figure(dpi = 288, figsize=plotHelperLatex.figSizer(1,2))
for i in range(len(d_offset)):
    plt.plot(np.linspace(n_steps-1,0,n_steps), GaussCorrection[i], label = r"%.1f $\sigma$" %d_offset[i])
plt.plot([0,n_steps-1], [cFactorAnalytic,cFactorAnalytic], label="analytical correction", linestyle="dotted", color = 'red')
plt.xlabel(r"$\mathrm{log_2}$ of discrete point density per $\sigma$ / 1")
plt.ylabel("Correction factor / 1")
plt.legend()
plt.show()

# Plotting of data

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from scipy import constants as const
import numpy as np
import plotHelperLatex
plotHelperLatex.setMatplotSettings()

umPerPx = 6.949
dumPerPx = 4E-2
dRelUmPerPx = dumPerPx/umPerPx

def wavToEnergy(lamb):
    #Energy in eV, lamb in nm
    return const.h*const.c/lamb*1e9/const.elementary_charge

def energyToWav(E):
    #Energy in eV, lamb in nm
    return const.h*const.c/E*1e9/const.elementary_charge

def paramDictatTime(param_dict, time_fs):
    return param_dict['A1']*np.exp(-time_fs/param_dict['tau1']) + param_dict['A2']*np.exp(-time_fs/param_dict['tau2'])
#import data from excel (terrible choice I know, but I want the visualisation and comparability)

def ErrorCorrectionConvoluted(signal, correctionFactor, peakRadiance, dRelUmPerPx, dRelPower):
    '''correctionfactor is already divided with peak radiance\\
    dRelPower = dPower/Power'''
    dPeakRadiance = dRelPower*peakRadiance + 2*peakRadiance*dRelUmPerPx
    return abs(signal*correctionFactor/peakRadiance*dPeakRadiance)

excel_path = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\CorrectedPump653ProbeWavelengthScan_UV-extended.xlsx"
SHG_wav_timescan = pd.read_excel(excel_path, sheet_name = "2024WavelengthScanParameters", skiprows=[0,1])
SHG_main = pd.read_excel(excel_path, sheet_name='2024WavelengthScan', skiprows=[0,1])
#print(SHG_main)
#print(SHG_main.keys())


wavelengths = SHG_main['Probe wavelength / nm']
#simple correction without map
dOD_SHG_main = SHG_main['cor_dOD / (mOD*m^2/W)']
#correction with map
dOD_SHG_mapCorr = SHG_main['hypothetical correction']
dOD_map_reference = SHG_main['mapReference']
dPumpPower = SHG_main['dRelPumpPower']
peakRad = SHG_main['Pump power density / (W/m^2)']



scan_wavs = SHG_wav_timescan['Probe wavelength / nm']
scan_corrFactors = SHG_wav_timescan['Correction Factor / (W/m^2)^-1']
scan_params = {
    'A1' : SHG_wav_timescan['A1'],
    'A2' : SHG_wav_timescan['A2'],
    'tau1' : SHG_wav_timescan['tau1'],
    'tau2' : SHG_wav_timescan['tau2'],
}

times = np.array([1e2, 1e4, 2e4, 4e4])

#print(ErrorCorrectionConvoluted(dOD_SHG_main, scan_corrFactors, peakRad, dRelUmPerPx, dPumpPower))

fig, ax = plt.subplots(1,1, figsize=plotHelperLatex.figSizer(1,2), dpi = 288)
'''
ax.plot(wavelengths, dOD_SHG_main, 'ro', label='standard correction', ls = "None")
ax.plot(wavelengths, dOD_SHG_mapCorr, 'bx', label='map corrected', ls = "None")
#ax.errorbar(wavelengths, dOD_SHG_main, yerr = ErrorCorrectionConvoluted(dOD_SHG_main, scan_corrFactors, peakRad, dRelUmPerPx, dPumpPower), ls = "None", capsize =2)
'''
for ind, time in enumerate(times):
    ax.plot(scan_wavs, dOD_map_reference*paramDictatTime(scan_params, time), '1', label="%.1f ps" %(time*1e-3))
''''''
ax.set_xlabel('probe wavelength / nm')
ax.set_ylabel(r'$\mathrm{\Delta OD / a.u.}$')

ax.set_xticks(np.arange(300, 701, 50))
secax = ax.secondary_xaxis('top', functions=(wavToEnergy, energyToWav))
secax.set_xticks(np.round(wavToEnergy(np.array([440, 500, 653])), 2))
secax.set_xlabel(r"$\mathrm{E_{probe} / eV}$")
ax.legend()
ax.grid()


times = np.array([1e2, 1e5])
colors = ['b', 'r']
fig2, ax2 = plt.subplots(1,1, figsize=plotHelperLatex.figSizer(1,2), dpi = 288)
'''
ax.plot(wavelengths, dOD_SHG_main, 'ro', label='standard correction', ls = "None")
ax.plot(wavelengths, dOD_SHG_mapCorr, 'bx', label='map corrected', ls = "None")
#ax.errorbar(wavelengths, dOD_SHG_main, yerr = ErrorCorrectionConvoluted(dOD_SHG_main, scan_corrFactors, peakRad, dRelUmPerPx, dPumpPower), ls = "None", capsize =2)
'''

ind_color2 = ['royalblue', 'lightcoral']
for ind, time in enumerate(times):
    ax2.plot(scan_wavs, dOD_map_reference*paramDictatTime(scan_params, time), colors[ind]+'x', label="map corrected: %.1f ps" %(times[ind]*1e-3))
    ax2.plot(scan_wavs, paramDictatTime(scan_params, time), color = ind_color2[ind], marker= '+', linestyle="None", label="assuming homogeniety: %.1f ps" %(times[ind]*1e-3))
''''''
ax2.set_xlabel('probe wavelength / nm')
ax2.set_ylabel(r'$\mathrm{\Delta OD / a.u.}$')

ax2.set_xticks(np.arange(300, 701, 50))
secax = ax2.secondary_xaxis('top', functions=(wavToEnergy, energyToWav))
secax.set_xticks(np.round(wavToEnergy(np.array([440, 500, 653])), 2))
secax.set_xlabel(r"$\mathrm{E_{probe} / eV}$")
ax2.legend()
ax2.grid()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import plotHelperLatex
plotHelperLatex.setMatplotSettings()

#copied from above
def wavToEnergy(lamb):
    #Energy in eV, lamb in nm
    return const.h*const.c/lamb*1e9/const.elementary_charge

def energyToWav(E):
    #Energy in eV, lamb in nm
    return const.h*const.c/E*1e9/const.elementary_charge


def paramDictatTime(param_dict, time_fs):
    return param_dict['A1']*np.exp(-time_fs/param_dict['tau1']) + param_dict['A2']*np.exp(-time_fs/param_dict['tau2'])
#import data from excel (terrible choice I know, but I want the visualisation and comparability)

def ErrorCorrectionConvoluted(signal, correctionFactor, peakRadiance, dRelUmPerPx, dRelPower):
    '''correctionfactor is already divided with peak radiance\\
    dRelPower = dPower/Power'''
    dPeakRadiance = dRelPower*peakRadiance + 2*peakRadiance*dRelUmPerPx
    return abs(signal*correctionFactor/peakRadiance*dPeakRadiance)

excel_path = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\CorrectedPump653ProbeWavelengthScan_UV-extended.xlsx"
SHG_wav_timescan = pd.read_excel(excel_path, sheet_name = "2024WavelengthScanParameters", skiprows=[0,1])
SHG_main = pd.read_excel(excel_path, sheet_name='2024WavelengthScan', skiprows=[0,1])
#print(SHG_main)
#print(SHG_main.keys())


wavelengths = SHG_main['Probe wavelength / nm']
#simple correction without map
dOD_SHG_main = SHG_main['cor_dOD / (mOD*m^2/W)']
#correction with map
dOD_SHG_mapCorr = SHG_main['hypothetical correction']
dOD_map_reference = abs(SHG_main['mapReference'])
dPumpPower = SHG_main['dRelPumpPower']
peakRad = SHG_main['Pump power density / (W/m^2)']


scan_wavs = SHG_wav_timescan['Probe wavelength / nm']
scan_corrFactors = SHG_wav_timescan['Correction Factor / (W/m^2)^-1']
scan_params = {
    'A1' : SHG_wav_timescan['A1'],
    'A2' : SHG_wav_timescan['A2'],
    'tau1' : SHG_wav_timescan['tau1'],
    'tau2' : SHG_wav_timescan['tau2'],
}


#high res zheng

zheng_high_res_path = r"C:\Users\M\Documents\Books\masterprojectinformation\images\Zheng2020_higherPrecisionRip.csv"
Zheng = pd.read_csv(zheng_high_res_path, sep=",")

ZhengDict = {}
ZhengDict['10ps'] = np.array([Zheng["10 ps"][1:], Zheng["Y10 ps"][1:]], dtype=float)
ZhengDict['200fs'] = np.array([Zheng["0.2 ps"][1:], Zheng["Y0.2 ps"][1:]], dtype=float)
ZhengDict['2500fs'] = np.array([Zheng["2.5 ps"][1:], Zheng["Y2.5 ps"][1:]], dtype=float)
ZhengDict['100ps'] = np.array([Zheng["100 ps"][1:], Zheng["Y100 ps"][1:]], dtype=float)
#print(np.array(ZhengDict["2500fs"][0,:],dtype=int))
#print(np.array(ZhengDict["2500fs"][0,:],dtype=int) == 680)
ZhengModifier = abs(ZhengDict["2500fs"][1,np.array(ZhengDict["2500fs"][0,:],dtype=int) == 679])

fig, axs = plt.subplots(1,1, figsize = plotHelperLatex.figSizer(1,2), dpi=288)

times = [2e2, 2.5e3, 1e4, 1e5]
colors = ['red', 'green', 'orange', 'royalblue']

indexMeasureModifier = np.array(scan_wavs, int) == 680
modifier_scan_params = {
    'A1' : SHG_wav_timescan['A1'][indexMeasureModifier],
    'A2' : SHG_wav_timescan['A2'][indexMeasureModifier],
    'tau1' : SHG_wav_timescan['tau1'][indexMeasureModifier],
    'tau2' : SHG_wav_timescan['tau2'][indexMeasureModifier],
}
measurementModifier = abs(dOD_map_reference[indexMeasureModifier]*paramDictatTime(modifier_scan_params, times[0]))[0]
#print((dOD_map_reference[indexMeasureModifier]*paramDictatTime(scan_params, time))[0])

for ind, key in enumerate(ZhengDict):
    axs.plot(ZhengDict[key][0,:], ZhengDict[key][1,:]/ZhengModifier, linestyle="None", marker = ".", color=colors[ind])

for ind, time in enumerate(times):
    axs.plot(scan_wavs, dOD_map_reference*paramDictatTime(scan_params, time)/measurementModifier, '1', color = colors[ind], label="%.1f ps" %(time*1e-3))

secax = axs.secondary_xaxis('top', functions=(wavToEnergy, energyToWav))
secax.set_xticks(np.round(wavToEnergy(np.array([440, 500, 653])), 2))
secax.set_xlabel(r"$\mathrm{E_{probe} / eV}$")

axs.set_xlabel('probe wavelength / nm')
axs.set_ylabel(r'relative $\Delta OD$ / a.u.')
axs.legend()
plt.grid(True)
plt.show()

In [ ]:
#this here is supposed to show 'raw' data over wavelength
#copied most from above
import pandas as pd
from matplotlib import pyplot as plt
from scipy import constants as const
import numpy as np
import plotHelperLatex
plotHelperLatex.setMatplotSettings()
import fileParsingMethods
import SeriesDegradation
import json

umPerPx = 6.949
dumPerPx = 4E-2
dRelUmPerPx = dumPerPx/umPerPx

def wavToEnergy(lamb):
    #Energy in eV, lamb in nm
    return const.h*const.c/lamb*1e9/const.elementary_charge

def energyToWav(E):
    #Energy in eV, lamb in nm
    return const.h*const.c/E*1e9/const.elementary_charge

def paramDictatTime(param_dict, time_fs):
    return param_dict['A1']*np.exp(-time_fs/param_dict['tau1']) + param_dict['A2']*np.exp(-time_fs/param_dict['tau2'])
#import data from excel (terrible choice I know, but I want the visualisation and comparability)

def ErrorCorrectionConvoluted(signal, correctionFactor, peakRadiance, dRelUmPerPx, dRelPower):
    '''correctionfactor is already divided with peak radiance\\
    dRelPower = dPower/Power'''
    dPeakRadiance = dRelPower*peakRadiance + 2*peakRadiance*dRelUmPerPx
    return abs(signal*correctionFactor/peakRadiance*dPeakRadiance)

#doing this manually because I hate myself, but not enough to automate it
#[directory, filename, fitJSON for t0]
filePaths = []
#STD
filePaths.append([r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\STD-680", r"saturation_2024-01-16_14-09.mat",  r"TAfitPump653Probe680_DecayCorrected.JSON", r"SummaryPump653Probe680.JSON"])
filePaths.append([r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\STD-520", r"saturation_2024-01-16_15-21.mat",  r"TAfitPump653Probe520_DecayCorrected.JSON", r"SummaryPump653Probe520.JSON"])
filePaths.append([r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\STD-502", r"saturation_2024-01-16_14-51.mat",  r"TAfitPump653Probe502_DecayCorrected.JSON", r"SummaryPump653Probe502.JSON"])
filePaths.append([r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\STD-493", r"saturation_2024-01-19_14-24.mat",  r"TAfitPump653Probe493_DecayCorrected.JSON", r"SummaryPump653Probe493Caution.JSON"])
#SHG
filePaths.append([r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\SHG-468", r"saturation_2024-01-16_18-38.mat",  r"TAfitPump653Probe468_DecayCorrected.JSON", r"SummaryPump653Probe468.JSON"])
filePaths.append([r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\SHG-460", r"saturation_2024-01-18_17-34.mat",  r"TAfitPump653Probe460_DecayCorrected.JSON", r"SummaryPump653Probe460Caution.JSON"])
filePaths.append([r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\SHG-450", r"saturation_2024-01-18_11-28.mat",  r"TAfitPump653Probe450_DecayCorrected.JSON", r"SummaryPump653Probe450.JSON"])
filePaths.append([r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\SHG-440", r"saturation_2024-01-16_19-07.mat",  r"TAfitPump653Probe440_DecayCorrected.JSON", r"SummaryPump653Probe440.JSON"])
filePaths.append([r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\SHG-430", r"saturation_2024-01-18_10-52.mat",  r"TAfitPump653Probe430_DecayCorrected.JSON", r"SummaryPump653Probe430.JSON"])
filePaths.append([r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\SHG-420", r"saturation_2024-01-17_14-46.mat",  r"TAfitPump653Probe420_DecayCorrected.JSON", r"SummaryPump653Probe420.JSON"])
filePaths.append([r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\SHG-410", r"saturation_2024-01-19_13-41.mat",  r"TAfitPump653Probe410_DecayCorrected.JSON", r"SummaryPump653Probe410Caution.JSON"])
filePaths.append([r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\SHG-400", r"saturation_2024-01-17_15-26.mat",  r"TAfitPump653Probe400_DecayCorrected.JSON", r"SummaryPump653Probe400.JSON"])
filePaths.append([r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\SHG-390", r"saturation_2024-01-18_12-03.mat",  r"TAfitPump653Probe390_DecayCorrected.JSON", r"SummaryPump653Probe390.JSON"])
filePaths.append([r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\SHG-380", r"saturation_2024-01-17_16-31.mat",  r"TAfitPump653Probe380_2_DecayCorrected.JSON", r"SummaryPump653Probe380_2.JSON"])
filePaths.append([r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\SHG-370", r"saturation_2024-01-17_17-11.mat",  r"TAfitPump653Probe370_DecayCorrected.JSON", r"SummaryPump653Probe370.JSON"])
filePaths.append([r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\SHG-360", r"saturation_2024-01-18_15-50.mat",  r"TAfitPump653Probe360_DecayCorrected.JSON", r"SummaryPump653Probe360_2.JSON"])
filePaths.append([r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\SHG-350", r"saturation_2024-01-18_16-39.mat",  r"TAfitPump653Probe350_DecayCorrected.JSON", r"SummaryPump653Probe350.JSON"])

#import map correction
excel_path = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\CorrectedPump653ProbeWavelengthScan_UV-extended.xlsx"
SHG_main = pd.read_excel(excel_path, sheet_name='2024WavelengthScan', skiprows=[0,1], usecols=["Probe wavelength / nm", "mapReference"])

SHG_main = SHG_main.sort_values(by="Probe wavelength / nm", ascending=False)
SHG_main.drop_duplicates()
SHG_main= SHG_main[:-2]
#print(SHG_main)


dOD_map_reference = SHG_main["mapReference"]
dOD_map_reference = abs(dOD_map_reference/np.mean(dOD_map_reference))

#now grab the data, correct/background substract it it and stuff it into an array as a mean
wav_vec = [680, 520, 502, 493, 468, 460, 450, 440, 430, 420, 410, 400, 390, 380, 370, 360, 350]
bgLen = np.ones(np.shape(wav_vec), dtype=int)*25
#the linear background ones need more precise backgroundlengths
bgLen[[0,1,2,6,8]] = [40, 30, 25, 35, 35]
data_raw = []
delays = []
t0 = []
for ind, path_list in enumerate(filePaths):
    #load json with data needed
    #overlap JSON
    json_file = open(path_list[0] + r"\\"[0] + path_list[3])
    overlapCorrection = json.load(json_file)["Correction Factor: per W/m^2 pump power"]
    #fit JSON
    json_file = open(path_list[0] + r"\\"[0] + path_list[2])
    temp_JSON = json.load(json_file)["entries"][0]
    
    #print(temp_JSON)
    if ('bgParam' in temp_JSON.keys()) and (temp_JSON['bgParam'][0] != 0):
        linBgBool = True
    else:
        linBgBool = False
    tempdata, _, degradeFactors, tempdelay, _ = SeriesDegradation.autoCompensation(path_list[1], dirPath = path_list[0], degConst = temp_JSON["tauExpDegrade"], p_ratio = temp_JSON["degradePowerRatio"], linearBg=linBgBool, backgroundMean=bgLen[ind])
    data_raw.append(np.mean(overlapCorrection*dOD_map_reference[ind]*tempdata*degradeFactors, axis = 0))
    t0.append(temp_JSON["startParameters"][0]["position"])
    #print(t0)
    delays.append(tempdelay- t0[ind])

delays = np.array(delays)
data_raw = np.array(data_raw)

#create my own average time delay with n steps per picosecond and calculate the average value for plotting
n_subticks = int(1)
plotDelay = np.linspace(-1e4, 3e4, 40*n_subticks)
plotData = np.zeros((len(wav_vec), len(plotDelay)))

for indWav in range(len(wav_vec)):
    for indDelay in range(len(plotDelay)):
        delay_boolArray = (delays[indWav,:] > plotDelay[indDelay]) & (delays[indWav,:] < plotDelay[indDelay] + 1e3/(n_subticks))
        #print(delay_boolArray)
        plotData[indWav,indDelay] = np.mean(data_raw[indWav, delay_boolArray], axis = None)
#print(data_raw)
plotData = plotData/np.max(abs(plotData))
delayGrid, wavGrid = np.meshgrid(plotDelay, np.arange(0,len(wav_vec), 1))

fig, ax = plt.subplots(1,1, figsize=plotHelperLatex.figSizer(1), dpi=288)
map = ax.pcolor(delayGrid*1e-3, wavGrid, plotData, cmap="bwr", clim=(-1,+1))
fig.colorbar(map, ax = ax, label=r"relative $\Delta$OD / a.u.")
ax.set_yticks(np.arange(0,len(wav_vec), 1), wav_vec)
ax.set_ylabel('probe wavelength / nm')
ax.set_xlabel('delay / ps')
plt.show()


### peak radiance variation

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from scipy import constants as const
from scipy import io
import numpy as np
import json

import plotHelperLatex
plotHelperLatex.setMatplotSettings()

#From 2023.12.22 PumpPowerVar in pump 653 Probe 680
#2nd-x;170.98460380387596;4.882207633023595;1.0;0.0;3.5228
#2nd-x Error;0.044586461622659335;0.10565856496080137;60.88881374774401;1.876611874015875;3.5228
#2nd-y;123.76193757659699;5.398671742339542;1.0;0.0;3.5228
#2nd-y Error;0.007988920725724555;0.018944810621290376;10.67173147785706;0.31277821947196766;3.5228

file = open(r"c:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\STD-680\2023.12.22_PumpPowerVar\TAfitPump653Probe680_OUTPUT.JSON")
entries = json.load(file)['entries']
entries_files = []
dOD = []
for entry in entries:
    entries_files.append(entry['inputFile'])
    dOD.append([entry['popt'][2], np.sqrt(entry['pcov'][2][2])])

dOD = np.array(dOD)
dOD[dOD == np.inf] = np.NaN
#print(dOD)
#these are in order
pumpRad = np.array([[620, 43], [620, 43], [620, 43], [3250, 30], [4430,30], [5900, 30], [4430, 30], [4430, 30], [4430, 30]])
probeRad = np.array([[910, 100], [1820, 60], [460, 100], [460, 100], [460, 100], [460, 100], [910, 60], [3640, 20], [1820, 30]])


#try calculating a fun correction factor based on the first 3 measurements that have identical pump radiance
#need to take the times such as done in degradation
io.loadmat(r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\STD-680\2023.12.22_PumpPowerVar\saturation_2023-12-22_11-47.mat")
#print(np.polyfit(pumpRad[2:6,0], dOD[2:6,0], 1, cov=True))
linParameter = np.polyfit(pumpRad[2:5,0], dOD[2:5,0], 1)
#print(pumpRad[])
#Plot first 6 points as they should have relatively little sample degradation, because of their order and the pump power
fig, ax = plt.subplots(1,2, figsize=plotHelperLatex.figSizer(1,2.6), dpi = 144)
plt.tight_layout()
#ax[0].plot(pumpRad[:6,0], dOD[:6,0], 'b.')
#yerror of dOD fitting is not sensible
ax[0].plot(pumpRad[(0,5),0], np.polyval(linParameter, pumpRad[(0,5),0]), 'royalblue', linestyle="dashed")
ax[0].errorbar(pumpRad[:6,0], dOD[:6,0], xerr = pumpRad[:6,1], ls = "None", ecolor = 'b', capsize = 2)

ax[0].set_xlabel(r'pump peak radiance / $\mathrm{\frac{W}{m^2}}$')
ax[0].set_ylabel('raw absorbance / mOD')
ax[0].set_ylim([0,-2.2])
#ax[1].plot(probeRad[6:,0], dOD[6:,0], 'b')
ax[1].errorbar(probeRad[6:,0], dOD[6:,0], xerr = probeRad[6:,1], ls="None", ecolor = "b", capsize = 2)
ax[1].set_xlabel(r'probe peak radiance / $\mathrm{\frac{W}{m^2}}$')


#including degradation
import SeriesDegradation
dirPath = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\STD-680\2023.12.22_PumpPowerVar"
filePaths = []
start = 6615
stop = 6620
filePaths += SeriesDegradation.numberFileGenerator(start, stop)
start = 6630
stop = 6632
filePaths += SeriesDegradation.numberFileGenerator(start, stop)
start = 6636
stop = 6644
filePaths += SeriesDegradation.numberFileGenerator(start, stop)

p_ratio = 590/1580
powerDensitiesIn = [1.05 for i in range(6)]
powerDensitiesIn = np.concatenate((powerDensitiesIn, [5.5 for i in range(3)][:]), axis = None)
powerDensitiesIn = np.concatenate((powerDensitiesIn, [7.5 for i in range(3)][:]), axis = None)
powerDensitiesIn = np.concatenate((powerDensitiesIn, [10 for i in range(3)][:]), axis = None)
powerDensitiesIn = np.array(powerDensitiesIn)*p_ratio
timesPump = SeriesDegradation.getTimes(filePaths, dirPath)


#get delta t of times
timesFromZeroPump = SeriesDegradation.parseTime(timesPump)
CorrectionEstimatedPump = SeriesDegradation.degradationCompensation(14100, timesFromZeroPump, 1, powerDensities=powerDensitiesIn)

corr_dODpumpVar = dOD[:6,0][:]*CorrectionEstimatedPump[1:-1:3].T
corr_dODpumpVar = corr_dODpumpVar[0]
ax[0].plot(pumpRad[:6,0], corr_dODpumpVar, "r.", linestyle="None")
pumpVar_corrFit = np.polyfit(pumpRad[:6,0], corr_dODpumpVar, deg = 2)
ax[0].plot(pumpRad[[0,5],0], np.polyval(pumpVar_corrFit, pumpRad[[0,5],0]), "orange", linestyle="dashed")
### Probe power var
filePaths = []
start = 6645
stop = 6653
filePaths += SeriesDegradation.numberFileGenerator(start, stop)

timesProbe = SeriesDegradation.getTimes(filePaths, dirPath)
#get delta t of times
timesFromZeroProbe = SeriesDegradation.parseTime(timesProbe)
CorrectionEstimatedProbe = SeriesDegradation.degradationCompensation(14100, timesFromZeroProbe, 1, powerDensities=7.5*p_ratio)
corr_dODprobeVar = dOD[6:,0][:]*CorrectionEstimatedProbe[1:-1:3].T
corr_dODprobeVar = corr_dODprobeVar[0]
ax[1].plot(probeRad[6:,0], corr_dODprobeVar, 'r.', linestyle="None")

#fig.suptitle('no degradation correction yet')
plt.show()

In [ ]:
import sys
import plotHelperLatex
plotHelperLatex.setMatplotSettings()
sys.path.insert(1, r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm")
from ShowDelayScan import main

#this is the constant pump power and probe power variation
filenames=[]
for x in range(6645,6654,1):
    filenames.append("\TA_fourier_%4d" %(x))
dirPath = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\STD-680\2023.12.22_PumpPowerVar"
main(filenames, dirPath, plotHelperLatex.figSizer(2), False)

filenames=[]
for x in range(6615,6621,1):
    filenames.append("\TA_fourier_%4d" %(x))
for x in range(6630,6633,1):
    filenames.append("\TA_fourier_%4d" %(x))
main(filenames, dirPath, plotHelperLatex.figSizer(2), False)

### Degradation

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from scipy import constants as const
from scipy import io 
import os
import numpy as np
from datetime import timedelta
import json
from scipy.optimize import curve_fit

import plotHelperLatex
plotHelperLatex.setMatplotSettings()
import SeriesDegradation

    

testArray = [r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\Pump653Probe493_Degradation\TA_fourier_6778.mat", 
             r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\Pump653Probe493_Degradation\TA_fourier_6779.mat"]

#to be tested yet
dArray, delay, _ = SeriesDegradation.parseSummaryFileToArray(r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\Pump653Probe493_Degradation\saturation_2024-01-19_16-14.mat")


#powerVar = pd.read_excel("CorrectedPump653ProbeWavelengthScan_UV-extended.xlsx", sheet_name='Degradation493nm')
dtime = SeriesDegradation.getTimes(r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\Pump653Probe493_Degradation\saturation_2024-01-19_16-14.mat")
dtime = SeriesDegradation.parseTime(dtime)
#dOD = powerVar['dOD / mOD']

dArray, aArray, _ = SeriesDegradation.removeBackground(dArray, 20)

t0_estimate = np.argmax((aArray[0,1:]-aArray[0,:-1])/(delay[1:]-delay[:-1]))
print(t0_estimate)
popt, pcov = SeriesDegradation.fitDegradation(aArray, dtime, t0=t0_estimate)

print(popt)
print(np.sqrt(pcov))


timeAt = np.array([0,1e3,5e3]) #fs


Corr = SeriesDegradation.degradationCompensation(popt, dtime, np.shape(aArray)[1])
fig, ax = plt.subplots(1,1, figsize=plotHelperLatex.figSizer(1,3), dpi = 288)
for i in range(len(timeAt)):
    ax.plot(dtime, aArray[:,i], label="delay = %.1f ps" %(timeAt[i]*1e-3))
    ax.plot(dtime, aArray[:,i]*Corr[:,i])
#ax.plot(dtime, dOD[0,2]-dtime*popt)
ax.legend()
ax.set_xlabel('time / s')
ax.set_ylabel('absorbance / mOD')

plt.show()

In [ ]:
import SeriesDegradation
import numpy as np
from matplotlib import pyplot as plt
import plotHelperLatex
plotHelperLatex.setMatplotSettings()


def numberFileGenerator(start, stop, namePrefix=r"TA_fourier_", nameSuffix=r""):
    ''''generate TA_fourier_02364 style names in list form'''
    vec = np.arange(start, stop+1,1)
    filenames = []
    for number in vec:
        filenames.append(namePrefix + str(number)+ nameSuffix)
    return filenames


start = 6778
stop = 6787
dirPath = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\Pump653Probe493_Degradation"


#got to fix the input to the changed plotTrend function
SeriesDegradation.plotTrend(numberFileGenerator(start, stop), dirPath, plotHelperLatex.figSizer(1,2))

In [ ]:
import SeriesDegradation
import numpy as np
from matplotlib import pyplot as plt
import plotHelperLatex
plotHelperLatex.setMatplotSettings()
import ShowDelayScan

#need to switch getTimes to the same system as parseSummary to allow any type of input
if False:
    start = 6778
    stop = 6787
    dirPath = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\Pump653Probe493_Degradation"
else:
    start = 6645
    stop = 6653
    dirPath = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\STD-680\2023.12.22_PumpPowerVar"


filePaths = SeriesDegradation.numberFileGenerator(start, stop)
print(len(filePaths))
times = SeriesDegradation.getTimes(filePaths, dirPath)
dArray, delay,_ = SeriesDegradation.parseSummaryFileToArray(filePaths, dirPath)
_, aArray, _ = SeriesDegradation.removeBackground(dArray, 10)
#get delta t of times
timesFromZero = np.array(SeriesDegradation.parseTime(times),dtype=int)

fig2, ax2 = plt.subplots(1,1, figsize=plotHelperLatex.figSizer(2.5), dpi = 144)
fig3, ax3 = plt.subplots(1,1, figsize=plotHelperLatex.figSizer(2.5), dpi = 144)
for ind in range(np.shape(aArray)[0]):
    ax2.plot(delay*1e-3, aArray[ind])

#estimate t0 by finding maximum value in aArray
t0_estimate = np.argmax((aArray[0,1:]-aArray[0,:-1])/(delay[1:]-delay[:-1]))
print(t0_estimate)
popt, pcov = SeriesDegradation.fitDegradation(aArray, timesFromZero, t0 = t0_estimate+2, sliding_window_len=5)
print(popt)
print(np.sqrt(pcov))

Correction = SeriesDegradation.degradationCompensation(popt, timesFromZero, np.shape(aArray)[1], powerDensities=1)
#Correction = SeriesDegradation.degradationCompensation(14000, timesFromZero, np.shape(aArray)[1], powerDensities=2.8)
CorrArray = np.zeros(np.shape(aArray))
for ind in range(np.shape(aArray)[0]):
    CorrArray[ind,:] = aArray[ind,:]*Correction[ind,:]
print(np.shape(aArray))
#timesFromZero = np.append(timesFromZero, np.mean(timesFromZero[1:]-timesFromZero[:-1], axis = None))
delays, measurementTimes = np.meshgrid(delay, timesFromZero)
fig, ax = plt.subplots(1,1, figsize=(8,4), dpi = 200)
map = ax.pcolor(delays*1e-3, measurementTimes, CorrArray, cmap="plasma", snap=False)
ax.set_xlabel('delay time / ps')
ax.set_ylabel('total time elapsed at start / s')
fig.colorbar(map, ax=ax, label=r"$\Delta OD$ / a.u.")
ax.set_yticks(timesFromZero-np.mean(timesFromZero[1]-timesFromZero[0], axis=None)/2, timesFromZero)
plt.tight_layout()


for ind in range(np.shape(aArray)[0]):
    ax3.plot(delay*1e-3, CorrArray[ind])
ax2.set_xlabel('delay time / ps')
ax2.set_ylabel('dOD / mOD')
ax3.set_xlabel('delay time / ps')
ax3.set_ylabel('dOD / mOD')
plt.show()

In [ ]:
#doing a comparison of powervar here
import SeriesDegradation
import numpy as np
from matplotlib import pyplot as plt
import plotHelperLatex
plotHelperLatex.setMatplotSettings()
import ShowDelayScan
from fileParsingMethods import numberFileGenerator


start = 6645
stop = 6653
dirPath = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\STD-680\2023.12.22_PumpPowerVar"


filePaths = numberFileGenerator(start, stop)
times = SeriesDegradation.getTimes(filePaths, dirPath)
dArray, delay, _ = SeriesDegradation.parseSummaryFileToArray(filePaths, dirPath)
_, aArray, _ = SeriesDegradation.removeBackground(dArray, 10)
#get delta t of times
timesFromZero = SeriesDegradation.parseTime(times)
fig1, ax1 = plt.subplots(1,1, figsize=plotHelperLatex.figSizer(3.2), dpi = 144)
fig2, ax2 = plt.subplots(1,1, figsize=plotHelperLatex.figSizer(3.2), dpi = 144)
fig3, ax3 = plt.subplots(1,1, figsize=plotHelperLatex.figSizer(3.2), dpi = 144)
for ind in range(np.shape(aArray)[0]):
    ax2.plot(delay*1e-3, aArray[ind])

#estimate t0 by finding maximum value in aArray
t0_estimate = np.argmax((aArray[0,1:]-aArray[0,:-1])/(delay[1:]-delay[:-1]))
print(t0_estimate)
popt, pcov = SeriesDegradation.fitDegradation(aArray, timesFromZero, t0 = t0_estimate+5, sliding_window_len=5)
print(popt)
print(np.sqrt(pcov))


CorrectionNative = SeriesDegradation.degradationCompensation(popt, timesFromZero, np.shape(aArray)[1], powerDensities=1)
CorrectionEstimated = SeriesDegradation.degradationCompensation(14100, timesFromZero, np.shape(aArray)[1], powerDensities=2.8)
CorrArray = np.zeros(np.shape(aArray))
for ind in range(np.shape(aArray)[0]):
    ax1.plot(delay*1e-3, aArray[ind,:])
    ax2.plot(delay*1e-3, aArray[ind,:]*CorrectionNative[ind,:])
    ax3.plot(delay*1e-3, aArray[ind,:]*CorrectionEstimated[ind,:])


#fig1.suptitle("raw")
fig2.suptitle("Native")
fig3.suptitle("Estimate")
ax1.set_xlabel('delay time / ps')
ax1.set_ylabel('dOD / mOD')
ax2.set_xlabel('delay time / ps')
ax2.set_ylabel('dOD / mOD')
ax3.set_xlabel('delay time / ps')
ax3.set_ylabel('dOD / mOD')
plt.show()

In [ ]:
#doing a comparison of powervar here
import SeriesDegradation
import numpy as np
from matplotlib import pyplot as plt
import plotHelperLatex
plotHelperLatex.setMatplotSettings()
import ShowDelayScan
from fileParsingMethods import numberFileGenerator



dirPath = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\STD-680\2023.12.22_PumpPowerVar"
filePaths = []
start = 6615
stop = 6620
filePaths += numberFileGenerator(start, stop)
start = 6630
stop = 6632
filePaths += numberFileGenerator(start, stop)
start = 6636
stop = 6644
filePaths += numberFileGenerator(start, stop)

p_ratio = 590/1580
powerDensitiesIn = [1.05 for i in range(6)]
powerDensitiesIn = np.concatenate((powerDensitiesIn, [5.5 for i in range(3)][:]), axis = None)
powerDensitiesIn = np.concatenate((powerDensitiesIn, [7.5 for i in range(3)][:]), axis = None)
powerDensitiesIn = np.concatenate((powerDensitiesIn, [10 for i in range(3)][:]), axis = None)
powerDensitiesIn = np.array(powerDensitiesIn)*p_ratio
times = SeriesDegradation.getTimes(filePaths, dirPath)
dArray, delay, _ = SeriesDegradation.parseSummaryFileToArray(filePaths, dirPath)
_, aArray, _ = SeriesDegradation.removeBackground(dArray, 10)

#get delta t of times
timesFromZero = SeriesDegradation.parseTime(times)
fig1, ax1 = plt.subplots(1,1, figsize=plotHelperLatex.figSizer(2), dpi = 288)
#fig2, ax2 = plt.subplots(1,1, figsize=plotHelperLatex.figSizer(3.2), dpi = 144)
#fig3, ax3 = plt.subplots(1,1, figsize=plotHelperLatex.figSizer(3.2), dpi = 144)


#estimate t0 by finding maximum value in aArray
t0_estimate = np.argmax((aArray[0,1:]-aArray[0,:-1])/(delay[1:]-delay[:-1]))
print(t0_estimate)
#popt, pcov = SeriesDegradation.fitDegradation(aArray, timesFromZero, t0 = t0_estimate+2, sliding_window_len=5)
#print(popt)
#print(np.sqrt(pcov))


CorrectionEstimated = SeriesDegradation.degradationCompensation(14100, timesFromZero, np.shape(aArray)[1], powerDensities=powerDensitiesIn)
#CorrectionEstimated = SeriesDegradation.degradationCompensation(14100, timesFromZero, np.shape(aArray)[1], powerDensities=2.8)
CorrArray = np.zeros(np.shape(aArray))
for ind in range(np.shape(aArray)[0]):
    ax1.plot(delay*1e-3, aArray[ind,:]*CorrectionEstimated[ind,:])
    #ax2.plot(delay*1e-3, aArray[ind,:]*CorrectionNative[ind,:])
    #ax3.plot(delay*1e-3, aArray[ind,:]*CorrectionEstimated[ind,:])


#fig1.suptitle("raw")
#fig2.suptitle("Native")
#fig3.suptitle("Estimate")
ax1.set_xlabel('delay time / ps')
ax1.set_ylabel('dOD / mOD')
#ax2.set_xlabel('delay time / ps')
#ax2.set_ylabel('dOD / mOD')
#ax3.set_xlabel('delay time / ps')
#ax3.set_ylabel('dOD / mOD')
plt.show()

### Correction value check

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from scipy import constants as const
import numpy as np
from datetime import timedelta
import sys
import json
sys.path.insert(1, r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm")
from peakRadianceAuto import autoPeakRadiance
from ArtrayAnalysis import ArtFit
import plotHelperLatex
plotHelperLatex.setMatplotSettings()
from matplotlib import ticker

def correctionFactorWithError(pathJSON, fittingPath):
    '''Returns correction factor with uncertainty in dOD per W/m^2\\
        pathJSON is to summary\\
            fittingPath is to standard fit.txt (2 beam pointspreads supported)'''
    #don't care about loading these multiple times
    #returns first 
    peakRad, dPeakRad = autoPeakRadiance(pathJSON, fittingPath)
    #only choose pump
    peakRad = peakRad[1]
    dPeakRad = dPeakRad[1]
    #dict with {"pump/probe": {"x/y" : [[sig, error],...]}}
    dataDict = ArtFit.parseFitFile(fittingPath)

    pump = dataDict["pump"]
    probe = dataDict["probe"]
    #coeffs = x_center, sig, Amplitude, offset
    #sig1Error = lambda corr, sig1, sig2, muDiff, dsig1 : dsig1*corr*(sig1**3/sig2**2+sig1+muDiff**2*sig1/(sig1**2+sig2**2)**2)
    #sig2Error = lambda corr, sig1, sig2, muDiff, dsig2: dsig2*corr*((sig1**2+sig2**2)/sig2 + muDiff**2*sig2/(sig1**2+sig2**2)**2)
    #muDiffError = lambda corr, sig1, sig2, muDiff, dmuDiff: dmuDiff*corr*muDiff/(sig1**2+sig2**2)

    sig1Error = lambda corr, coeffs1, coeffs2: coeffs1[1,1]*corr*(coeffs1[1,0]**3/coeffs2[1,0]**2+coeffs1[1,0]+(coeffs1[0,0]-coeffs2[0,0])**2*coeffs1[1,0]/(coeffs1[1,0]**2+coeffs2[1,0]**2)**2)
    sig2Error = lambda corr, coeffs1, coeffs2: coeffs2[1,1]*corr*((coeffs1[1,0]**2+coeffs2[1,0]**2)/coeffs2[1,0]+ (coeffs1[0,0]-coeffs2[0,0])**2*coeffs2[1,0]/(coeffs1[1,0]**2+coeffs2[1,0]**2)**2)
    #using compound error for this one
    muDiffError = lambda corr, coeffs1, coeffs2: (coeffs1[0,1]+coeffs2[0,1])*corr*(coeffs1[0,0]-coeffs2[0,0])/(coeffs1[1,0]**2+coeffs2[1,0]**2)
    #simplification
    fullError = lambda corr, coeffs1, coeffs2: sig1Error(corr, coeffs1, coeffs2) + sig2Error(corr, coeffs1, coeffs2) + muDiffError(corr, coeffs1, coeffs2)
    #takes pump then probe
    xCorr = ArtFit.calculateOverlapCorrection(pump["x"][:,0], probe["x"][:,0], 5)
    dXCorr = fullError(xCorr, pump['x'], probe['x'])
    yCorr = ArtFit.calculateOverlapCorrection(pump["y"][:,0], probe["y"][:,0], 5)
    dYCorr = fullError(xCorr, pump['y'], probe['y'])
    Corr = [xCorr, yCorr]
    totalCorr = xCorr*yCorr
    totalError = (xCorr*dYCorr+yCorr*dXCorr)/peakRad + totalCorr*dPeakRad/peakRad**2
    #print(totalCorr)
    #print(totalError)
    sigCalc = lambda probeCoeff, pumpCoeff: [probeCoeff[1,0]/pumpCoeff[1,0], probeCoeff[1,1]/pumpCoeff[1,0] + probeCoeff[1,0]/pumpCoeff[1,0]**2*pumpCoeff[1,1]]
    sigbysig = np.array([sigCalc(probe['x'], pump['x']), sigCalc(probe['y'], pump['y'])], dtype = float)
    dmu = np.array([[abs(probe["x"][0,0]-pump["x"][0,0]), probe["x"][0,1]+pump["x"][0,1]],[abs(probe["y"][0,0]-pump["y"][0,0]), probe["y"][0,1]+pump["y"][0,1]]], dtype = float)
    return [Corr, totalError], sigbysig, dmu, pump, probe, peakRad



file = open(r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\Pump653Probe493_ArtrayCompensationControl\TAfitPump653Probe493_OUTPUT.JSON")
entries = json.load(file)['entries']
entries_files = []
dOD = []
for entry in entries:
    entries_files.append(entry['inputFile'])
    dOD.append([entry['popt'][2]+ entry['popt'][4], entry['pcov'][2][2]+entry['pcov'][4][2]])

dOD = np.array(dOD)
dOD[dOD == np.inf] = np.NaN



basepath = lambda filename: r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\Pump653Probe493_ArtrayCompensationControl" + filename
inputFiles = [(r"\SummaryPump653Probe493Caution.JSON", r"\BeamFitsPump653Probe493Caution.txt"),
              (r"\SummaryPump653Probe493_ShiftedProbe.JSON", r"\BeamFitsPump653Probe493_ShiftedProbe.txt"),
              (r"\SummaryPump653Probe493_WideProbe.JSON", r"\BeamFitsPump653Probe493_WideProbe.txt"),
              (r"\SummaryPump653Probe493_ThinProbe.JSON", r"\BeamFitsPump653Probe493_ThinProbe.txt")]
labels = ["standard", "shifted", "wide probe", "thin probe"]
correctionFactors = np.zeros((4,2))
correctionFactors = []
dmu = np.zeros((4,2,2))
sigbysig = np.zeros((4,2,2))
pump_Coeffs =[]
probe_Coeffs = []



for index, shortPath in enumerate(inputFiles):
    tempCorrectionFactors, sigbysig[index], dmu[index], pump_temp, probe_temp, peakRad = correctionFactorWithError(basepath(shortPath[0]), basepath(shortPath[1]))
    correctionFactors.append(tempCorrectionFactors)
    print("xsigRatio: %.3e +- %.3e, xdmu: %.3e +- %.3e" %(sigbysig[index, 0,0], sigbysig[index, 0,1], dmu[index, 0,0], dmu[index, 0,1]))
    print("ysigRatio: %.3e +- %.3e, ydmu: %.3e +- %.3e" %(sigbysig[index,1,0], sigbysig[index,1,1], dmu[index,1,0], dmu[index,1,1]))
    print("%s: %.3e +- %.3e dOD per W/msquare" %(labels[index], correctionFactors[index][0][0]*correctionFactors[index][0][1]*dOD[index, 0]/peakRad, correctionFactors[index][1]*dOD[index, 0]))
    pump_Coeffs.append(pump_temp)
    probe_Coeffs.append(probe_temp)
#plotting
fig, axs = plt.subplots(3,4, figsize = plotHelperLatex.figSizer(0.9,2.5), dpi = 288)

#doing all 4 measurements horizontally
def contourPlot(axis_handle, coeffs1, coeffs2, x, y, lvl = 5, colormap ="viridis"):
    #x, y are x and y vectors to be used
    "coeffs = center, sig, Amplitude, offset"
    [X_contour, Y_contour] = np.meshgrid(x, y)
    Z_contour = ArtFit.gaussian(X_contour, coeffs1)*ArtFit.gaussian(Y_contour, coeffs2)

    #adjust for correct height
    Z_contour = Z_contour/np.sqrt(coeffs1[2]*coeffs2[2])
    axis_handle.contour(X_contour, Y_contour, Z_contour, levels = lvl, cmap = colormap)
    axis_handle.set_aspect('equal')

ticker.MaxNLocator(integer = True)

def textGenerator(axis_handle, sig, dmu, correctionFactors):
    step= -0.15*3
    axis_handle.text(0,0, r"x: $\Delta \mu$=%.1f px" %dmu[0,0])
    axis_handle.text(0,step, r"$\frac{\sigma_{probe}}{\sigma_{pump}}$=%.2f" %sig[0,0])
    axis_handle.text(0,2*step, r"Correction: %.2f" %correctionFactors[0])
    axis_handle.text(0,3*step, r"y: $\Delta \mu$=%.1f px" %dmu[1,0])
    axis_handle.text(0,4*step, r"$\frac{\sigma_{probe}}{\sigma_{pump}}$=%.2f" %sig[1,0])
    axis_handle.text(0,5*step, r"Correction: %.2f" %correctionFactors[1])
    


#do legend on top
axs[0,0].text(0.5,0, "pump pointspread", verticalalignment ="center")
axs[0,0].plot(0,0, marker = 's', color = "purple")
axs[0,0].text(0.5,0.05, "  probe pointspread", verticalalignment ="center")
axs[0,0].plot(0,0.05, marker = 's', color = "green")
axs[0,0].set_xlim((-0.5,4.5))
axs[0,0].set_ylim((-0.05,0.1))

for index, shortPath in enumerate(inputFiles):
    #remove legend axis
    axs[0,index].set_axis_off()
    axs[2,index].set_axis_off()
    axs[0,index].set_adjustable('box')
    #find x and y vectors to be used
    x = (pump_Coeffs[index]['x'][0,0] + probe_Coeffs[index]['x'][0,0])/2
    y = (pump_Coeffs[index]['y'][0,0] + probe_Coeffs[index]['y'][0,0])/2
    sigMax = max([pump_Coeffs[index]['x'][1,0], pump_Coeffs[index]['y'][1,0], probe_Coeffs[index]['x'][1,0], probe_Coeffs[index]['y'][1,0]])
    x = np.linspace(x-2*sigMax, x+2*sigMax, 100)
    y = np.linspace(y-2*sigMax, y+2*sigMax, 100)
    #plot contour pump
    contourPlot(axs[1, index], pump_Coeffs[index]['x'][:,0], pump_Coeffs[index]['y'][:,0], x, y, colormap="Purples")
    #plot contour probe
    contourPlot(axs[1, index], probe_Coeffs[index]['x'][:,0], probe_Coeffs[index]['y'][:,0], x, y, colormap="Greens")
    axs[1,index].tick_params(axis='both', which='major', labelsize=7)
    axs[1,index].set_xticks([180, 185])
    axs[1,index].set_yticks([165, 170])
    textGenerator(axs[2,index], sigbysig[index], dmu[index], correctionFactors[index][0])
    #axs[2, index].text(0,0, "test")
#plt.tight_layout()
plt.show()


In [ ]:
import pandas
from matplotlib import pyplot as plt
import numpy as np 

import plotHelperLatex
plotHelperLatex.setMatplotSettings()

import sys
import json
sys.path.insert(1, r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm")
from peakRadianceAuto import autoPeakRadiance
from ArtrayAnalysis import ArtFit
#plot the graphs as contour, but not from fits
#basing this on rerunFromJSON classmethod of ArtFit
def countourComparisonRaw(axes_handle, JSONpath, x, y, bgSubtract = False):
    '''currently only for full pump and probe files, implement only single beam later
    currently always uses backgroundfile scale'''
    print(JSONpath)
    f_JSON = open(JSONpath, 'r')
    summary_JSON = json.load(f_JSON)
    f_JSON.close()
    Routine = ArtFit(summary_JSON["BackgroundJSON"], summary_JSON["pathProbe"], summary_JSON["pathPump"], summary_JSON["FitAvgWidth / px"])
    probeMap = Routine.probeMap - Routine.bgMap
    pumpMap = Routine.pumpMap - Routine.bgMap
    
    probeMap = np.transpose(probeMap[y[0]:y[-1]+1, x[0]:x[-1]+1])
    pumpMap = np.transpose(pumpMap[y[0]:y[-1]+1, x[0]:x[-1]+1])
    Xgrid, Ygrid= np.meshgrid(y, x)
    #plot probe
    axes_handle.contour(Ygrid, Xgrid, probeMap, levels = 5, cmap = "Greens")
    #plot pump
    axes_handle.contour(Ygrid, Xgrid, pumpMap, levels = 5, cmap = "Purples")



basepath = lambda filename: r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\Pump653Probe493_ArtrayCompensationControl" + filename
inputFiles = [(r"\SummaryPump653Probe493Caution.JSON", r"\BeamFitsPump653Probe493Caution.txt"),
              (r"\SummaryPump653Probe493_ShiftedProbe.JSON", r"\BeamFitsPump653Probe493_ShiftedProbe.txt"),
              (r"\SummaryPump653Probe493_WideProbe.JSON", r"\BeamFitsPump653Probe493_WideProbe.txt"),
              (r"\SummaryPump653Probe493_ThinProbe.JSON", r"\BeamFitsPump653Probe493_ThinProbe.txt")]

pump_Coeffs = []
probe_Coeffs = []
for index, shortPath in enumerate(inputFiles):
    dataDict = ArtFit.parseFitFile(basepath(inputFiles[index][1]))
    pump_Coeffs.append(dataDict["pump"])
    probe_Coeffs.append(dataDict["probe"])

fig, axs = plt.subplots(1,4, dpi=144, figsize=plotHelperLatex.figSizer(1,4))
#taken from plot above
for index, shortPath in enumerate(inputFiles):
    #find x and y vectors to be used
    x = (pump_Coeffs[index]['x'][0,0] + probe_Coeffs[index]['x'][0,0])/2
    y = (pump_Coeffs[index]['y'][0,0] + probe_Coeffs[index]['y'][0,0])/2
    sigMax = max([pump_Coeffs[index]['x'][1,0], pump_Coeffs[index]['y'][1,0], probe_Coeffs[index]['x'][1,0], probe_Coeffs[index]['y'][1,0]])
    x = np.arange(x-2.5*sigMax, x+2.5*sigMax, 1, dtype=int)
    y = np.arange(y-2.5*sigMax, y+2.5*sigMax, 1, dtype=int)
    #plot both contours
    countourComparisonRaw(axs[index], basepath(inputFiles[index][0]), x, y)
    axs[index].tick_params(axis='both', which='major', labelsize=7)
    #axs[index].set_xticks([180, 185])
    #axs[index].set_yticks([165, 170])
plt.tight_layout()



In [ ]:
#test plotting product of 4 gaussians
import numpy as np
from matplotlib import pyplot as plt

def gaussian(x, popt):
    #center, sig
    center = popt[0]
    sig = popt[1]
    return 1/np.sqrt(2*np.pi)/sig*np.exp(-(center-x)**2/(2*sig**2))
    
#coeffs for gaussian
"coeffs = x_center, sig, Amplitude, offset"
alpha = 15/np.pi
coeffsX_1 = [0, 10]
coeffsX_2 = [0, 6]
coeffsY_1 = [0, 10]
coeffsY_2 = [0, 6]
x = np.linspace(-30, 30, 100)
y = np.linspace(-30, 30, 100)
X_grid, Y_grid = np.meshgrid(x, y)

Z = gaussian()
plt.figure()
plt.contour()






### 440 vs orthogonal

In [ ]:
import numpy as np
from fileParsingMethods import numberFileGenerator
import sys
sys.path.insert(1, r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm")
import ShowDelayScan

#reference responses for 680 are identical at -1.17 mOD

fileNames = numberFileGenerator(6692,6696)
dirPath = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\SHG-440\Orthogonal"
ShowDelayScan.main(fileNames, dirPath, showLegend= True)

summaryParallel = r"C:\Users\M\Documents\phdmatlab\sqib-pmma-probe-wavelength\UV_Setup\new_parallel_pol_pump653nm\SHG-440\saturation_2024-01-16_19-07.mat"
ShowDelayScan.main(summaryParallel)